In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as random
from casadi import *
from pdfo import *

In [2]:
n = 500
taux = 1/n
T = np.linspace(0,1,n)

In [3]:
def tracer_orientation (x,y,theta, r, i):
    if i == 1 :
        plt.arrow(x, y, r*cos(theta),r*sin(theta), width = 0.01, color = 'red' , label = "Axe local suivant x")
        plt.arrow(x, y, r*cos(pi/2+theta),r*sin(pi/2+theta), width = 0.01, color = 'yellow' , label = "Axe local suivant y")
        plt.legend()
    else :
        plt.arrow(x, y, r*cos(theta),r*sin(theta), width = 0.01, color = 'red' )
        plt.arrow(x, y, r*cos(pi/2+theta),r*sin(pi/2+theta), width = 0.01, color = 'yellow' )
 

In [15]:
######################################## Bi-level PDFO

#bounds = np.array([[0, 1], [0, 1], [0, 1], [0, 1],[0, 1], [0, 1],[0, 1], [0, 1], [0, 1], [0, 1],[0, 1], [0, 1]])

options = {'maxfev': 10000 , 'rhobeg' : 0.1 , 'rhoend' : 1e-8}


Lin_const = []

for i in range(n):
    Lin_const.append(LinearConstraint([1, t[i], (t**2)[i],(t**3)[i],(t**4)[i],(t**5)[i],1, t[i], (t**2)[i],(t**3)[i],(t**4)[i],(t**5)[i]], 1, 1))
    Lin_const.append(LinearConstraint([0, 0,0,0,0,0,1, t[i], (t**2)[i],(t**3)[i],(t**4)[i],(t**5)[i]], 0, 1))    
    Lin_const.append(LinearConstraint([1, t[i], (t**2)[i],(t**3)[i],(t**4)[i],(t**5)[i],0,0,0,0,0,0], 0, 1))    

In [1]:
def Unicycle (C) :
    [A0,A1,A2,A3,A4,A5,B0,B1,B2,B3,B4,B5] = C
    c1 = A0* (T**0) + A1* (T**1) + A2* (T**2) + A3* (T**3) + A4* (T**4) + A5* (T**5) 
    c2 = B0* (T**0) + B1* (T**1) + B2* (T**2) + B3* (T**3) + B4* (T**4) + B5* (T**5)
    print(C)
    
    c01 = c1.copy()
    c02 = c2.copy()
    
    mk = 0
    
    for j in range (c1.shape[0]):
        if c1[j] < 0 :
            c01[j] = - c1[j] 
            mk = mk - c1[j] 
        if c2[j] < 0 :
            c02[j] = - c2[j]
            mk = mk - c2[j] 
            
    
    opti = casadi.Opti()   # cette fonction nous permet de trouver la solution de problème

    u1 = opti.variable(n)
    u2 = opti.variable(n)
    x1 = opti.variable(n)
    x2 = opti.variable(n)
    x3 = opti.variable(n)

    opti.minimize(   (taux/2)*(dot(c01*u1,u1)+dot(c02*u2,u2))   )    # ma fonction objetion

    # mes fonctions de contrainte d'égalité:

    ## pour les condition initial
    opti.subject_to( x1[0]==X1[0] + 10**(-4))        
    opti.subject_to( x2[0]==X2[0] + 10**(-4))
    opti.subject_to( x3[0]==X3[0] + 10**(-4))
    
    opti.subject_to( u1[0] == 0.0001 )
    opti.subject_to( u2[0] == 0.0001 )
    opti.subject_to( u1[-1] == 0.0001)
    opti.subject_to( u2[-1] == 0.0001)
    

    ## pour les contraintes d'égaliter
    opti.subject_to( u1[:n-1]*cos(x3[:n-1])==(x1[1:] + 10**(-4) - x1[:n-1])/taux)
    opti.subject_to( u1[:n-1]*sin(x3[:n-1])==(x2[1:] + 10**(-4) - x2[:n-1])/taux)
    opti.subject_to( u2[:n-1] ==(x3[1:] + 10**(-4) - x3[:n-1])/taux)
    
    ## pour les conditions finales
    opti.subject_to( x1[-1]==X1[-1] + 10**(-4))
    opti.subject_to( x2[-1]==X2[-1] + 10**(-4))
    opti.subject_to( x3[-1]==X3[-1] + 10**(-4))
    
    opti.solver('ipopt', {"expand":True}, {"max_iter":10000})
    
    sol = opti.solve() 
    
    X1_1 = opti.debug.value(x1)
    X2_1 = opti.debug.value(x2)
    X3_1 = opti.debug.value(x3)
    
    plt.plot(X1_1,X2_1, color = 'green')
    
    m01 = sqrt((np.linalg.norm(X1-X1_1)**2 + np.linalg.norm(X2-X2_1)**2 + np.linalg.norm(X3-X3_1)**2 )/n)
    
    m02 = 10*abs(sum(c1 + c2) - n)
    
    m03 = 10* mk
    
    m1 = m01+m02+m03
    
    return m1
